In [1]:
import os
import cv2
import tqdm
import copy
import json
import random
import numpy as np
import skimage as sk
from scipy import ndarray
from scipy import ndimage
from PIL import Image


def rotate(image_array: ndarray, angle):
    return ndimage.rotate(image_array, angle, reshape=True, order=0)


def brightness(img, brightness):
    return sk.exposure.adjust_gamma(img, brightness).astype(np.uint8)


def resize(img, scale_percent):
    return cv2.resize(
        img,
        (
            int(img.shape[1] * scale_percent / 100),
            int(img.shape[0] * scale_percent / 100)
        ),
        interpolation = cv2.INTER_AREA
    )


def get_and_adjust_rotation_matrix(angle, cx, cy, h, w):
    M = cv2.getRotationMatrix2D((cx, cy), angle, 1.0)
    cosang = np.abs(M[0, 0])
    sinang = np.abs(M[0, 1])
    nW = int((h * sinang) + (w * cosang))
    nH = int((h * cosang) + (w * sinang))
    M[0, 2] += (nW / 2) - cx
    M[1, 2] += (nH / 2) - cy
    return M


def rotatePolygon(corners, M, w, h, ln=8):
    corners = corners.reshape(-1,2)
    corners = np.hstack((corners, np.ones((corners.shape[0],1), dtype = type(corners[0][0]))))
    calculated = np.dot(M,corners.T).T
    calculated = calculated.reshape(-1,ln)
    if len(np.where(calculated[:, range(0,ln,2)] > w)) == 0 or len(np.where(calculated[:, range(1,ln,2)] > h)) == 0:
        return None
    return calculated


def transform_bboxes_for_resize_for_angle(bboxes, rotation_matrix, w, h):
    new_bboxes = {}
    for label, bbox in bboxes.items():
        p = [
            [bbox[0],bbox[1]],
            [bbox[2],bbox[1]],
            [bbox[2],bbox[3]],
            [bbox[0],bbox[3]]
        ]
        nl = np.array(p).reshape(-1,)
        p1 = rotatePolygon(np.array([nl]), rotation_matrix, w, h, ln=len(nl))[0]
        p1 = [(p1[i],p1[i+1]) for i in range(0,len(nl),2)]
        new_bboxes[label] = p1
    return new_bboxes


def transform_bboxes_for_resize(img, bboxes, scale_percent):
    return {
        k: [
            int(v[0] * scale_percent / 100),
            int(v[1] * scale_percent / 100),
            int(v[2] * scale_percent / 100),
            int(v[3] * scale_percent / 100)
        ]
        for k,v in bboxes.items()
    }


def augment_img_n_bbox(img, sharpness, angle, scale_percent, bboxes):
    augmented = copy.deepcopy(img)
    transformed_bboxes = copy.deepcopy(bboxes)
    transformed_bboxes = transform_bboxes_for_resize(augmented, transformed_bboxes, scale_percent)
    augmented = resize(img, scale_percent)
    cx = augmented.shape[1]//2
    cy = augmented.shape[0]//2
    rotation_matrix = get_and_adjust_rotation_matrix(angle, cx, cy, augmented.shape[0], augmented.shape[1])
    transformed_bboxes = transform_bboxes_for_resize_for_angle(transformed_bboxes, rotation_matrix, augmented.shape[1], augmented.shape[0])    
    augmented = rotate(augmented, angle)
    augmented = brightness(augmented, sharpness)
    return augmented, transformed_bboxes


def load_bboxes(json_path, im_dir_path):
    data = json.load(open(json_path,"r"))
    bboxes = {}
    for k, v in data["_via_img_metadata"].items():
        im_path = os.path.join(im_dir_path,v["filename"])
        if os.path.exists(im_path):
            bboxes[im_path] = {}
            for r in v["regions"]:
                bboxes[im_path][list(r["region_attributes"]["class"])[0]] = [
                    r["shape_attributes"]["x"],
                    r["shape_attributes"]["y"],
                    r["shape_attributes"]["x"]+r["shape_attributes"]["width"],
                    r["shape_attributes"]["y"]+r["shape_attributes"]["height"]
                ]
    return bboxes


def bboxes_to_coco(bboxes):
    categories = []
    all_labels = {}
    images = []
    ann = []
    annotations = {
        "info": {
            "year": 2023,
            "version": '1',
            "description": "ML1",
            "contributor": "ML1",
            "url": "ml1.ai",
            "date_created": "Mon Mar 06 2023 00:00:00 GMT+0500 (Pakistan Standard Time)"
        },
        "licenses": [{"id": 1, "name": "Unknown", "url": ""}]
    }
    for im_path, boxes in  bboxes.items():
        im = cv2.imread(im_path)
        im_name = im_path.split("/")[-1]
        im_id = len(images)
        images += [
            {
                "id": im_id,
                "width": im.shape[1],
                "height": im.shape[0],
                "file_name": im_name,
                "license": 1,
                "date_captured": ""
            }
        ]
        for label,box in boxes.items():
            all_x = [int(b[0]) for b in box]
            all_y = [int(b[1]) for b in box]
            x1, x2, y1, y2 = min(all_x), max(all_x), min(all_y), max(all_y)
            b = [x1, y1, x2-x1, y2-y1]
            if label not in all_labels:
                all_labels[label] = len(all_labels) + 1
            ann += [
                {
                    "id": len(ann),
                    "category_id": all_labels[label],
                    "image_id": im_id,
                    "segmentation": [int(x )for x in np.array(box).reshape(-1)],
                    "area": b[2]*b[3],
                    "bbox": b,
                    "iscrowd": 0
                }
            ]
    annotations["images"] = images
    annotations["annotations"] = ann
    annotations["categories"] = [
        {"id": label_id, "name": label, "supercategory": 'class'}
        for label, label_id in all_labels.items()
    ]
    return annotations


def augment_and_convert(json_file_path, img_dir_path, dest_dir_path, angle_interval):
    if not os.path.exists(dest_dir_path):
        os.mkdir(dest_dir_path)

    annotations_data = load_bboxes(json_file_path, img_dir_path)

    new_bboxes = {}
    for im_path, bboxes in annotations_data.items():
        print("processing ("+str(len(new_bboxes))+"/"+str(len(annotations_data))+"): "+im_path)
        im_name = im_path.split("/")[-1]
        img = cv2.imread(im_path)
        if len(img.shape) is 2:
            img = gray2rgb(img)
        img = img[:,:,:3]
        for angle in tqdm.tqdm(range(0,360,angle_interval)):
            sharpness  = random.randint(100, 200)/100.0
            scale_percent = random.randint(60,100)
            augmented, transformed_bboxes = augment_img_n_bbox(img, sharpness, angle, scale_percent, bboxes)
            new_im_name = im_name.rsplit(".",1)[0] + "_" + str(angle) +  "_" + str(sharpness) + "_" + str(scale_percent) + ".png"
            new_im_path = os.path.join(dest_dir_path, new_im_name)
            cv2.imwrite(new_im_path, augmented)
            new_bboxes[new_im_path] = transformed_bboxes
    print("Augmentations done!\nSaving annotations ...")
    new_json_data = bboxes_to_coco(new_bboxes)
    json.dump(new_json_data, open(os.path.join(dest_dir_path,"annotations.json"),"w"))
    print("Done annotations.json in ",dest_dir_path)
    



<>:181: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:181: SyntaxWarning: "is" with a literal. Did you mean "=="?
/var/folders/qx/blgsg83d77d2zb7gf3v3b6s40000gq/T/ipykernel_96976/1801741132.py:181: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if len(img.shape) is 2:


In [2]:

json_file_path = "cat_dog_annotations.json"
img_dir_path = "new_cat_dog"

dest_dir_path = "destination"

angle_interval = 5

augment_and_convert(json_file_path, img_dir_path, dest_dir_path, angle_interval)



processing (0/7): /Users/waqas_temp/Downloads/cat_dog/1.jpeg


100%|██████████| 72/72 [00:07<00:00,  9.07it/s]


processing (72/7): /Users/waqas_temp/Downloads/cat_dog/2.jpeg


100%|██████████| 72/72 [00:02<00:00, 26.15it/s]


processing (144/7): /Users/waqas_temp/Downloads/cat_dog/3.jpeg


...

processing (360/7): /Users/waqas_temp/Downloads/cat_dog/6.jpeg


100%|██████████| 72/72 [00:02<00:00, 31.95it/s]


processing (432/7): /Users/waqas_temp/Downloads/cat_dog/7.jpeg


100%|██████████| 72/72 [00:03<00:00, 21.70it/s]


Augmentations done!
Saving annotations ...
Done annotations.json in  /Users/waqas_temp/Downloads/dest


In [3]:

# import copy
# from PIL import Image
# from PIL import ImageDraw
# from PIL import ImageFont


# def visualize(image, b_boxes, color="red"):
#     font_path='/Users/waqas_temp/Downloads/arial-unicode-ms.ttf'
#     font = ImageFont.truetype(font_path, 20)
#     img = copy.deepcopy(image)
#     if type(b_boxes) is dict:
#         draw = ImageDraw.Draw(img)
#         for label in b_boxes:
#             box=b_boxes[label]
#             draw.rectangle(((box[0], box[1]), (box[2], box[3])), outline=color, width=2)
#             draw.text((box[0], box[1]-22), label, fill=color, font=font)
#         return img
#     elif type(b_boxes) is list:
#         draw = ImageDraw.Draw(img)
#         for box in b_boxes:
#             draw.rectangle(((box[0], box[1]), (box[2], box[3])), outline=color, width=2)
#         return img
#     return img

# def visualize_p(image, b_boxes, color="red"):
#     font_path='/Users/waqas_temp/Downloads/arial-unicode-ms.ttf'
#     font = ImageFont.truetype(font_path, 20)
#     img = copy.deepcopy(image)
#     if type(b_boxes) is dict:
#         draw = ImageDraw.Draw(img)
#         for label in b_boxes:
#             box=b_boxes[label]
#             draw.polygon(box, outline=color, width=2)
#             draw.text((box[0][0], box[0][1]-22), label, fill=color, font=font)
            
#         return img
#     elif type(b_boxes) is list:
#         draw = ImageDraw.Draw(img)
#         for box in b_boxes:
#             draw.polygon(box, outline=color, width=2)
#         return img
#     return img


